In [210]:
import requests
import json
import pandas as pd
r = requests.get(url = 'https://api.sledilnik.org/api/stats')
resp_text = r.text
json_dict=json.loads(resp_text)

In [211]:
#print(json_dict[90])

In [212]:

dataset={}
dates = []
#positive = []
positive_by_age=dict()
for i in range(len(json_dict)):
    
    date = []
    day = json_dict[i]['day']
    month = json_dict[i]['month']
    year= json_dict[i]['year']

    age_positive = json_dict[i]['statePerAgeToDate']
    age_deceased = json_dict[i]['deceasedPerAgeToDate']

    #date.append(day)
    #date.append(month)
    #date.append(year)

    #print(date)
    date = str(day) + '/' + str(month) + '/' + str(year)
    
    #positive_by_age = dict()
    deceased_by_age = dict()
    tmp = dict()
    
    for j in range(len(age_positive)):
        age_group = str(age_positive[j]['ageFrom']) + '-' + str(age_positive[j]['ageTo'])
        age_group_total = age_positive[j]['allToDate']
        age_group_female = age_positive[j]['femaleToDate']
        age_group_male = age_positive[j]['maleToDate']
        tmp[age_group] = [age_group_total, age_group_male, age_group_female]
        #print(positive_by_age[age_group])
        
    positive_by_age[i] = tmp
        
    #for i in range(len(age_deceased)):
        #print (age_deceased[i])

    dates.append(date)

#print(positive_by_age)
dataset = {'Dates': dates, 
           '0-4 Total': [positive_by_age[i]['0-4'][0] for i in range(len(positive_by_age))], '0-4 Male':  [positive_by_age[i]['0-4'][1] for i in range(len(positive_by_age))], '0-4 Female':  [positive_by_age[i]['0-4'][2] for i in range(len(positive_by_age))], 
           '5-14 Total': [positive_by_age[i]['5-14'][0] for i in range(len(positive_by_age))], '5-14 Male': [positive_by_age[i]['5-14'][1] for i in range(len(positive_by_age))], '5-14 Female':[positive_by_age[i]['5-14'][2] for i in range(len(positive_by_age))],
           '15-24 Total': [positive_by_age[i]['15-24'][0] for i in range(len(positive_by_age))], '15-24 Male': [positive_by_age[i]['15-24'][1] for i in range(len(positive_by_age))], '15-24 Female': [positive_by_age[i]['15-24'][2] for i in range(len(positive_by_age))],
           '25-34 Total': [positive_by_age[i]['25-34'][0] for i in range(len(positive_by_age))], '25-34 Male': [positive_by_age[i]['25-34'][1] for i in range(len(positive_by_age))], '25-34 Female': [positive_by_age[i]['25-34'][2] for i in range(len(positive_by_age))],
           '35-44 Total': [positive_by_age[i]['35-44'][0] for i in range(len(positive_by_age))], '35-44 Male': [positive_by_age[i]['35-44'][1] for i in range(len(positive_by_age))], '35-44 Female': [positive_by_age[i]['35-44'][2] for i in range(len(positive_by_age))],
           '45-54 Total': [positive_by_age[i]['45-54'][0] for i in range(len(positive_by_age))], '45-54 Male':  [positive_by_age[i]['45-54'][1] for i in range(len(positive_by_age))], '45-54 Female':  [positive_by_age[i]['45-54'][2] for i in range(len(positive_by_age))],
           '55-64 Total': [positive_by_age[i]['55-64'][0] for i in range(len(positive_by_age))], '55-64 Male': [positive_by_age[i]['55-64'][1] for i in range(len(positive_by_age))], '55-64 Female': [positive_by_age[i]['55-64'][2] for i in range(len(positive_by_age))],
           '65-74 Total': [positive_by_age[i]['65-74'][0] for i in range(len(positive_by_age))], '65-74 Male': [positive_by_age[i]['65-74'][1] for i in range(len(positive_by_age))], '65-74 Female': [positive_by_age[i]['65-74'][2] for i in range(len(positive_by_age))],
           '75-84 Total': [positive_by_age[i]['75-84'][0] for i in range(len(positive_by_age))], '75-84 Male': [positive_by_age[i]['75-84'][1] for i in range(len(positive_by_age))], '75-84 Female': [positive_by_age[i]['75-84'][2] for i in range(len(positive_by_age))],
           '85+ Total': [positive_by_age[i]['85-None'][0] for i in range(len(positive_by_age))], '85+ Male': [positive_by_age[i]['85-None'][1] for i in range(len(positive_by_age))], '85+ Female': [positive_by_age[i]['85-None'][2] for i in range(len(positive_by_age))],
          
          }

In [213]:
#print(age_deceased[9])

In [214]:
#print(by_age_group)

In [215]:
#print(dataset)

In [216]:
import pandas as pd
  
df = pd.DataFrame(dataset) 
df

,Dates,0-4 Total,0-4 Male,0-4 Female,5-14 Total,5-14 Male,5-14 Female,15-24 Total,15-24 Male,15-24 Female,...,55-64 Female,65-74 Total,65-74 Male,65-74 Female,75-84 Total,75-84 Male,75-84 Female,85+ Total,85+ Male,85+ Female
0,24/2/2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,25/2/2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,26/2/2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,27/2/2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,28/2/2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
232,13/10/2020,67.0,33.0,34.0,446.0,230.0,216.0,1321.0,682.0,639.0,...,620.0,612.0,324.0,288.0,481.0,209.0,272.0,423.0,108.0,315.0
233,14/10/2020,71.0,34.0,37.0,492.0,253.0,239.0,1426.0,730.0,696.0,...,661.0,648.0,345.0,303.0,503.0,220.0,283.0,437.0,111.0,326.0
234,15/10/2020,82.0,40.0,42.0,535.0,277.0,258.0,1560.0,808.0,752.0,...,700.0,693.0,366.0,327.0,541.0,236.0,305.0,461.0,117.0,344.0
235,16/10/2020,88.0,42.0,46.0,587.0,304.0,283.0,1680.0,872.0,808.0,...,753.0,733.0,383.0,350.0,573.0,246.0,327.0,488.0,126.0,362.0
